# Debezium with Jupyter Notebook

The following example shows how you can run debezium in a jupyter notebook

In [4]:
//DEPS io.debezium:debezium-embedded:3.4.0-SNAPSHOT
//DEPS org.dflib:dflib-jupyter:2.0.0-M4
//DEPS io.debezium:debezium-connector-postgres:3.4.0-SNAPSHOT

import static java.lang.System.*;

import java.util.Map;
import java.util.concurrent.ExecutorService;
import java.util.concurrent.Executors;
import java.util.function.Function;

import org.apache.kafka.connect.source.SourceRecord;
import org.apache.kafka.connect.data.Struct;

import io.debezium.config.Configuration;
import io.debezium.embedded.Connect;
import io.debezium.engine.DebeziumEngine;
import io.debezium.engine.RecordChangeEvent;
import io.debezium.engine.format.ChangeEventFormat;
import io.debezium.pipeline.notification.SnapshotStatus;
import org.dflib.*;
import org.dflib.builder.DataFrameArrayAppender;

var config = Configuration.empty()
    .withSystemProperties(Function.identity())
    .edit()
    .with(Configuration.from(Map.of(
        "connector.class", "io.debezium.connector.postgresql.PostgresConnector",
        "name", "native",
        "database.hostname", "localhost",
        "database.port", "5432",
        "database.user", "native",
        "database.password", "native",
        "database.dbname", "native",
        "topic.prefix", "dbserver1",
        "offset.storage","org.apache.kafka.connect.storage.MemoryOffsetBackingStore",
        "snapshot.mode", "initial")))
    .with("table.include.list","demo.saas_metrics_monthly")
    .build();

DataFrameArrayAppender appender = DataFrame.byArrayRow("Month Date",
                    "MRR Total",
                    "Active Users",
                    "Free Users",
                    "Enterprise Users")
            .appender();

var engine = DebeziumEngine
    .create(ChangeEventFormat.of(Connect.class))
    .using(config.asProperties())
    .addShutdownStrategy(event -> event.equals(SnapshotStatus.TOTALLY_COMPLETED.name()))
                .notifying((record) -> {
                    Integer monthDate = ((Struct) ((Struct) record.record().value()).get("after")).getInt32("month_date");

                    LocalDate localDate = LocalDate.ofEpochDay(monthDate);
                    BigDecimal mrrTotal = (BigDecimal) ((Struct) ((Struct) record.record().value()).get("after")).get("mrr_total");
                    Integer activeUsers = ((Struct) ((Struct) record.record().value()).get("after")).getInt32("active_users");
                    Integer freeUsers = ((Struct) ((Struct) record.record().value()).get("after")).getInt32("users_free");
                    Integer usersEnterprise = ((Struct) ((Struct) record.record().value()).get("after")).getInt32("users_enterprise");

                    appender.append(localDate, mrrTotal, activeUsers, freeUsers, usersEnterprise);
                })
    .build();

engine.run();
var dataFrame = appender.toDataFrame();
display("## Extracted Data");
dataFrame;

JBangInfo{dependencies=3,resolvedDependencies=161,newResolvedDependencies=0}

## Extracted Data


Month Date MRR Total Active Users Free Users Enterprise Users
---------- --------- ------------ ---------- ----------------
2025-01-01    750.00          420        120               70
2025-02-01    980.00          470        130               80
2025-03-01   1200.00          520        140               90
2025-04-01   1490.00          580        150              100
2025-05-01   1780.00          640        160              120
...
2025-08-01   2480.00          790        180              190
2025-09-01   2790.00          850        185              215
2025-10-01   2990.00          910        190              250
2025-11-01   3250.00          960        195              275
2025-12-01   3510.00         1010        200              300
12 rows x 5 columns

## Bar chart: Monthly Recurring Revenue (MRR)

In [2]:
ECharts
        .chart()
        .xAxis("Month Date")
        .series(SeriesOpts.ofBar(), "MRR Total")
        .plot(dataFrame);

<script type='text/javascript' src='https://cdn.jsdelivr.net/npm/echarts@5.6.0/dist/echarts.min.js'></script><div id='dfl_ech_9265' class='dfl_ech' style='width: 600px;height:400px;'></div><script type='text/javascript'>var chart_dfl_ech_9265 = echarts.init(
        document.getElementById('dfl_ech_9265'),
        null,
   );
    var option_dfl_ech_9265 = {
        dataset: {
            source: [
                ['L0','2025-01-01','2025-02-01','2025-03-01','2025-04-01','2025-05-01','2025-06-01','2025-07-01','2025-08-01','2025-09-01','2025-10-01','2025-11-01','2025-12-01'],
                ['MRR Total',750.00,980.00,1200.00,1490.00,1780.00,1980.00,2210.00,2480.00,2790.00,2990.00,3250.00,3510.00]
            ]
        },
        xAxis: [
        {
            type: 'category'
        },
        ],
        yAxis: [
        {
            type: 'value'
        },
        ],
        series: [
            {
                name: 'MRR Total',
                encode: {
                    x: 0,
                    y: 1,
                },
                seriesLayoutBy: 'row',
                type: 'bar'
            },
        ]
    };
    option_dfl_ech_9265 && chart_dfl_ech_9265.setOption(option_dfl_ech_9265);</script>

In [3]:
ECharts
        .chart()
        .xAxis("Month Date")
        .series(SeriesOpts.ofBar(), "Active Users")
        .plot(dataFrame);

<script type='text/javascript' src='https://cdn.jsdelivr.net/npm/echarts@5.6.0/dist/echarts.min.js'></script><div id='dfl_ech_8784' class='dfl_ech' style='width: 600px;height:400px;'></div><script type='text/javascript'>var chart_dfl_ech_8784 = echarts.init(
        document.getElementById('dfl_ech_8784'),
        null,
   );
    var option_dfl_ech_8784 = {
        dataset: {
            source: [
                ['L0','2025-01-01','2025-02-01','2025-03-01','2025-04-01','2025-05-01','2025-06-01','2025-07-01','2025-08-01','2025-09-01','2025-10-01','2025-11-01','2025-12-01'],
                ['Active Users',420,470,520,580,640,690,730,790,850,910,960,1010]
            ]
        },
        xAxis: [
        {
            type: 'category'
        },
        ],
        yAxis: [
        {
            type: 'value'
        },
        ],
        series: [
            {
                name: 'Active Users',
                encode: {
                    x: 0,
                    y: 1,
                },
                seriesLayoutBy: 'row',
                type: 'bar'
            },
        ]
    };
    option_dfl_ech_8784 && chart_dfl_ech_8784.setOption(option_dfl_ech_8784);</script>